In [4]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv


In [6]:
import pandas as pd
import numpy as np
import sklearn
import tensorflow as tf

titanic_train = pd.read_csv ("train.csv")
titanic_test = pd.read_csv ("test.csv")

In [7]:
titanic_train = titanic_train.drop(['PassengerId','Name','Ticket','Cabin'],axis=1)
titanic_train = titanic_train.dropna(subset=['Embarked'])

titanic_test = titanic_test.drop(['PassengerId','Name','Ticket','Cabin'],axis=1)

titanic_train_2 = pd.get_dummies(data=titanic_train)
titanic_test_2 = pd.get_dummies(data=titanic_test)

In [8]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')
train_np = imputer.fit_transform(titanic_train_2)
test_np = imputer.fit_transform(titanic_test_2)

titanic_train_3 = pd.DataFrame(train_np)
titanic_test_3 = pd.DataFrame(test_np)

In [9]:
train_features = titanic_train_3.copy()
train_labels = train_features.pop(0)

In [10]:
from sklearn.model_selection import train_test_split
SEED=12345
x_train,x_test,y_train,y_test = train_test_split(train_features,train_labels,test_size=0.2,random_state=SEED)

x_train_np_1= np.array(x_train)
standardizer = sklearn.preprocessing.StandardScaler()

standardizer.fit(x_train)
x_train = standardizer.transform(x_train)
x_test = standardizer.transform(x_test)
wild_test = standardizer.transform(test_np)

In [11]:
#Data preparation is done
x_train_np_2 = np.array(x_train)

In [12]:
#Model creation
nClass = len(np.unique(y_test))
inputs = tf.keras.Input(shape=(x_train.shape[-1],))
dense = tf.keras.layers.Dense(64,activation='relu')
x = dense(inputs)
dense = tf.keras.layers.Dense(32,activation='relu')
x = dense(x)
dense = tf.keras.layers.Dense(16,activation='relu')
x = dense(x)
outputs = tf.keras.layers.Dense(nClass,activation='softmax')(x)

model = tf.keras.Model(inputs=inputs,outputs=outputs,name='titanic_model')
model.summary()

Model: "titanic_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10)]              0         
                                                                 
 dense (Dense)               (None, 64)                704       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dense_3 (Dense)             (None, 2)                 34        
                                                                 
Total params: 3,346
Trainable params: 3,346
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
history = model.fit(x_train,y_train,validation_data=(x_test,y_test),batch_size=16,epochs=25)

Epoch 1/25
45/45 [==============================] - 1s 8ms/step - loss: 0.5555 - accuracy: 0.7454 - val_loss: 0.4937 - val_accuracy: 0.7809
Epoch 2/25
45/45 [==============================] - 0s 3ms/step - loss: 0.4464 - accuracy: 0.8003 - val_loss: 0.4755 - val_accuracy: 0.7865
Epoch 3/25
45/45 [==============================] - 0s 3ms/step - loss: 0.4245 - accuracy: 0.8087 - val_loss: 0.4740 - val_accuracy: 0.7978
Epoch 4/25
45/45 [==============================] - 0s 3ms/step - loss: 0.4113 - accuracy: 0.8087 - val_loss: 0.4649 - val_accuracy: 0.8146
Epoch 5/25
45/45 [==============================] - 0s 3ms/step - loss: 0.3990 - accuracy: 0.8298 - val_loss: 0.4717 - val_accuracy: 0.8202
Epoch 6/25
45/45 [==============================] - 0s 3ms/step - loss: 0.3945 - accuracy: 0.8298 - val_loss: 0.4616 - val_accuracy: 0.8034
Epoch 7/25
45/45 [==============================] - 0s 3ms/step - loss: 0.3892 - accuracy: 0.8368 - val_loss: 0.4654 - val_accuracy: 0.8258
Epoch 8/25
45/45 [==

In [14]:
print(np.argmax(model.predict(wild_test),axis=-1))

[0 0 0 0 0 0 1 0 1 0 0 0 1 0 1 1 0 0 0 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 0 0
 1 0 0 1 0 1 1 1 0 0 0 0 1 0 1 1 0 0 1 0 1 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 1 0 0 1 1 1 1 0 0 0 1 0 1 0 0 0 0 0 0 1 1 1 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 1 0 1 0 1 0 0 0 1 1 1 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 0 0 1 0 0 0 1 0 0 1 0 1 0 1 1 1 1 0 0 0 0 1 0 1 0 1 0 1 0 0 0 0 0 1
 0 0 0 1 0 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 1 0 1 1 0 0 1 1 0
 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 1 0 1 0 0 0 0 1 1 0 1 0 0 1 1 0
 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 1 0 1 0 0 1 0 1 0 1 0 0
 1 1 0 1 1 0 0 1 0 0 0]
